In [16]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../filings.db')

queue_df = pd.read_sql_query("SELECT * FROM dispatch_queue", conn)

conn.close()

In [17]:
queue_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1505 entries, 0 to 1504
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1505 non-null   int64 
 1   adsh          1505 non-null   object
 2   form          1505 non-null   object
 3   cik           1505 non-null   object
 4   url           1505 non-null   object
 5   company_name  1505 non-null   object
 6   filing_date   1505 non-null   object
 7   filing_href   1505 non-null   object
 8   enqueued_ts   1505 non-null   object
 9   processed     1505 non-null   int64 
dtypes: int64(2), object(8)
memory usage: 117.7+ KB


In [18]:
# 2) Extract 8-K text with edgartools
# pip install edgar   # if not already installed

from edgar import find, set_identity
set_identity("Sri <sri.jerripotula@gmail.com>")
pd.set_option('future.no_silent_downcasting', True)
def extract_8k_text(adsh: str) -> str:
    try:
        filing = find(adsh)
        eightk = filing.obj()          
        parts = []
        for item in eightk.items:      
            parts.append(f"--- {item} ---\n{eightk[item]}")
        return "\n\n".join(parts)
    except Exception as e:
        return f"ERROR: {e}"
eight_k = queue_df[queue_df.form == '8-K']
mask_8k = eight_k['form'].eq('8-K')
eight_k.loc[mask_8k, 'text'] = eight_k.loc[mask_8k, 'adsh'].apply(extract_8k_text)
eight_k


C:\Users\sriha\AppData\Local\Temp\ipykernel_16828\3727779499.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eight_k.loc[mask_8k, 'text'] = eight_k.loc[mask_8k, 'adsh'].apply(extract_8k_text)


,id,adsh,form,cik,url,company_name,filing_date,filing_href,enqueued_ts,processed,text
1,2,0001213900-25-077711,8-K,0001832483,https://www.sec.gov/Archives/edgar/data/183248...,,,,2025-08-18 18:01:15,0,--- Item 3.02 ---\nItem 3.02. Unregistered Sal...
20,21,0001614178-25-000105,8-K,0001614178,https://www.sec.gov/Archives/edgar/data/161417...,,,,2025-08-18 18:01:15,0,--- Item 2.02 ---\nItem 2.02. Results of Opera...
29,30,0001062993-25-014530,8-K,0001498148,https://www.sec.gov/Archives/edgar/data/149814...,,,,2025-08-18 18:01:15,0,--- Item 8.01 ---\nItem 8.01 Other Events\n On...
45,46,0001104659-25-079521,8-K,0001039828,https://www.sec.gov/Archives/edgar/data/103982...,,,,2025-08-18 18:01:15,0,--- Item 2.02 ---\n\nItem 2.02 Results of Op...
52,53,0001104659-25-079459,8-K,0001720893,https://www.sec.gov/Archives/edgar/data/172089...,,,,2025-08-18 18:01:15,0,--- Item 8.01 ---\n\nItem 8.01 Other Events....
...,...,...,...,...,...,...,...,...,...,...,...
1388,1389,0001193125-25-184090,8-K,0000790051,https://www.sec.gov/Archives/edgar/data/790051...,,,,2025-08-21 02:57:54,0,--- Item 1.01 ---\n\nItem 1.01. Entry into a...
1389,1390,0001104659-25-080912,8-K,0001014739,https://www.sec.gov/Archives/edgar/data/101473...,,,,2025-08-21 02:57:54,0,--- Item 5.02 ---\nItem 5.02. Departure of Dir...
1394,1395,0001193125-25-184165,8-K,0000910638,https://www.sec.gov/Archives/edgar/data/910638...,,,,2025-08-21 02:57:54,0,--- Item 5.02 ---\nItem 5.02. Departure of Dir...
1423,1424,0000950170-25-110414,8-K,0002026738,https://www.sec.gov/Archives/edgar/data/202673...,,,,2025-08-21 02:57:54,0,--- Item 8.01 ---\nItem 8.01 Other Events.\nJu...


In [11]:
eight_k

,id,adsh,form,cik,url,company_name,filing_date,filing_href,enqueued_ts,processed,clickable_url
1,2,0001213900-25-077711,8-K,0001832483,https://www.sec.gov/Archives/edgar/data/183248...,,,,2025-08-18 18:01:15,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
20,21,0001614178-25-000105,8-K,0001614178,https://www.sec.gov/Archives/edgar/data/161417...,,,,2025-08-18 18:01:15,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
29,30,0001062993-25-014530,8-K,0001498148,https://www.sec.gov/Archives/edgar/data/149814...,,,,2025-08-18 18:01:15,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
45,46,0001104659-25-079521,8-K,0001039828,https://www.sec.gov/Archives/edgar/data/103982...,,,,2025-08-18 18:01:15,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
52,53,0001104659-25-079459,8-K,0001720893,https://www.sec.gov/Archives/edgar/data/172089...,,,,2025-08-18 18:01:15,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
...,...,...,...,...,...,...,...,...,...,...,...
1388,1389,0001193125-25-184090,8-K,0000790051,https://www.sec.gov/Archives/edgar/data/790051...,,,,2025-08-21 02:57:54,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
1389,1390,0001104659-25-080912,8-K,0001014739,https://www.sec.gov/Archives/edgar/data/101473...,,,,2025-08-21 02:57:54,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
1394,1395,0001193125-25-184165,8-K,0000910638,https://www.sec.gov/Archives/edgar/data/910638...,,,,2025-08-21 02:57:54,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."
1423,1424,0000950170-25-110414,8-K,0002026738,https://www.sec.gov/Archives/edgar/data/202673...,,,,2025-08-21 02:57:54,0,"<a href=""https://www.sec.gov/Archives/edgar/da..."


In [25]:
# from IPython.display import HTML
# HTML(eight_k[['company_name', 'text', 'url']].to_html(escape=False, index=False))

In [17]:
# pip install instructor pydantic>=2 openai

from openai import OpenAI
import instructor
from pydantic import BaseModel

class MaterialEvent(BaseModel):
    event_type: str
    signal: str
    confidence: float
    reasoning: str

client = instructor.from_openai(OpenAI(api_key=""))
filing_text = eight_k.text.iloc[4]
filing_text

event = client.chat.completions.create(
    model="gpt-4o-mini",  # <-- REQUIRED when using from_openai(...)
    response_model=MaterialEvent,
    messages=[
        {"role": "system", "content": """You are an expert financial analyst trained on SEC filings.
You will be given the text of an 8-K filing.

Your task is to detect whether the filing contains any material events that could impact investors or stock price, such as (but not limited to):

Bankruptcy or insolvency
Acquisition or disposal of a business
Changes in executive leadership (CEO, CFO, board members, etc.)
Large asset impairments or write-downs
Public-to-private or private-to-public company transitions
Modifications to shareholder rights
Other significant events of similar material importance (etc.)

Instructions:

Identify all material events (not just the ones listed above).
Do not hallucinate; only mark events if you can directly support them with quoted evidence.
Return structured JSON:

{
  "event_type": "string (one of: Bankruptcy, Acquisition/Disposal, ExecChange, Impairment, PublicPrivate, ShareholderRights)",
  "signal": "Bullish | Bearish | Neutral",
  "confidence": "float between 0 and 1",
  "reasoning": "short explanation for why this event impacts the signal"
}
"""},
        {"role": "user", "content": filing_text},
    ],
    temperature=0,
)

print(event.model_dump())


{'event_type': 'Other', 'signal': 'Bullish', 'confidence': 0.8, 'reasoning': 'The positive feedback from the FDA regarding the sNDA submission for BXCL501 indicates potential for future approval and market success, which is generally viewed positively by investors.'}


In [ ]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('../filings.db')

queue_df = pd.read_sql_query("SELECT * FROM dispatch_queue", conn)

conn.close()

In [6]:
# Display 8-K filings with clickable links
from IPython.display import HTML

# Filter for 8-K filings only
eight_k = queue_df[queue_df.form == "8-K"].copy()

# Create clickable links for the URL column
def make_clickable_link(url):
    if pd.isna(url) or not isinstance(url, str):
        return "No URL"
    return f'<a href="{url}" target="_blank" rel="noopener">View Filing</a>'

# Apply the clickable link function
eight_k['clickable_url'] = eight_k['url'].apply(make_clickable_link)

display_columns = ['company_name', 'filing_date', 'clickable_url', 'adsh']
HTML(eight_k[display_columns].to_html(escape=False, index=False))

company_name,filing_date,clickable_url,adsh
,,View Filing,0001213900-25-077711
,,View Filing,0001614178-25-000105
,,View Filing,0001062993-25-014530
,,View Filing,0001104659-25-079521
,,View Filing,0001104659-25-079459
,,View Filing,0001104659-25-079463
,,View Filing,0001641172-25-024525
,,View Filing,0001140361-25-031632
,,View Filing,0001079973-25-001337
,,View Filing,0001140361-25-031674


In [14]:
# Display SCHEDULE 13G filings with clickable links
from IPython.display import HTML

# Filter for SCHEDULE 13G filings only
schedule_13g = queue_df[queue_df.form == "SCHEDULE 13G"].copy()

# Create clickable links for the URL column
def make_clickable_link(url):
    if pd.isna(url) or not isinstance(url, str):
        return "No URL"
    return f'<a href="{url}" target="_blank" rel="noopener">View Filing</a>'

# Apply the clickable link function
schedule_13g['clickable_url'] = schedule_13g['url'].apply(make_clickable_link)

display_columns = ['company_name', 'filing_date', 'clickable_url', 'adsh']
HTML(schedule_13g[display_columns].to_html(escape=False, index=False))

company_name,filing_date,clickable_url,adsh
,,View Filing,0001213900-25-077794
,,View Filing,0000935836-25-000538
,,View Filing,0001493152-25-012118
,,View Filing,0000921895-25-002331
,,View Filing,0001214659-25-012492
,,View Filing,0001076809-25-000101
,,View Filing,0001213900-25-079093
,,View Filing,0001104659-25-080978


In [15]:
queue_df.form.unique()

array(['N-PX', '8-K', '6-K', '424B2', 'F-1/A', '497K', '485BPOS',
       'NPORT-P', '144', '13F-HR', 'D', '4', 'NT 10-Q', '13F-NT',
       '8-A12B', 'S-3ASR', 'DEFA14A', '497', 'SCHEDULE 13D', 'D/A',
       'CERT', 'FWP', '425', 'SCHEDULE 13G/A', 'S-6', 'N-8A', 'X-17A-5',
       'C/A', 'ABS-15G', 'SCHEDULE 13G', '10-Q', '485APOS',
       'SCHEDULE 13D/A', '25-NSE', '497J', 'N-1A', '424B3', '13F-HR/A',
       '485BXT', 'SC 14D9', 'C', 'DEFM14A', 'DEF 14C', 'DSTRBRPT',
       '8-K/A', '24F-2NT', '3', '5', 'DEF 14A', '6-K/A', 'SC 14D9/A',
       '424B5', 'SC TO-T', '487', 'ATS-N/UA', 'N-2/A', 'S-3/A', '497VPI',
       '424B4', '497VPU', '40-24B2', '253G2', 'N-23C3A', 'S-1', '10-Q/A',
       '10-K', 'POSASR', 'PRE 14A', 'N-PX/A', '486APOS', 'F-1', 'MA-I/A',
       'SC TO-I/A', '10-K/A', 'S-1MEF', '305B2', 'ABS-EE/A', 'N-6/A',
       'S-1/A', '1-U', 'N-CEN/A', 'SC TO-I', '8-K12B', 'RW', 'S-6/A',
       '40-17F2', 'ABS-EE', 'N-23C-2', '4/A', '10-D', 'N-VP/A', 'S-4',
       '497AD', 'N-CSRS',